In [1]:
#############################################
# Kural Tabanlı Sınıflandırma ile Potansiyel Müşteri Getirisi Hesaplama
#############################################

#############################################
# İş Problemi
#############################################
# Bir oyun şirketi müşterilerinin bazı özelliklerini kullanarak seviye tabanlı (level based) yeni müşteri tanımları (persona)
# oluşturmak ve bu yeni müşteri tanımlarına göre segmentler oluşturup bu segmentlere göre yeni gelebilecek müşterilerin şirkete
# ortalama ne kadar kazandırabileceğini tahmin etmek istemektedir.

# Örneğin: Türkiye’den IOS kullanıcısı olan 25 yaşındaki bir erkek kullanıcının ortalama ne kadar kazandırabileceği belirlenmek isteniyor.


#############################################
# Veri Seti Hikayesi
#############################################
# Persona.csv veri seti uluslararası bir oyun şirketinin sattığı ürünlerin fiyatlarını ve bu ürünleri satın alan kullanıcıların bazı
# demografik bilgilerini barındırmaktadır. Veri seti her satış işleminde oluşan kayıtlardan meydana gelmektedir. Bunun anlamı tablo
# tekilleştirilmemiştir. Diğer bir ifade ile belirli demografik özelliklere sahip bir kullanıcı birden fazla alışveriş yapmış olabilir.

# Price: Müşterinin harcama tutarı
# Source: Müşterinin bağlandığı cihaz türü
# Sex: Müşterinin cinsiyeti
# Country: Müşterinin ülkesi
# Age: Müşterinin yaşı

################# Uygulama Öncesi #####################

#    PRICE   SOURCE   SEX COUNTRY  AGE
# 0     39  android  male     bra   17
# 1     39  android  male     bra   17
# 2     49  android  male     bra   17
# 3     29  android  male     tur   17
# 4     49  android  male     tur   17

################# Uygulama Sonrası #####################

#       customers_level_based        PRICE SEGMENT
# 0   BRA_ANDROID_FEMALE_0_18  1139.800000       A
# 1  BRA_ANDROID_FEMALE_19_23  1070.600000       A
# 2  BRA_ANDROID_FEMALE_24_30   508.142857       A
# 3  BRA_ANDROID_FEMALE_31_40   233.166667       C
# 4  BRA_ANDROID_FEMALE_41_66   236.666667       C


#############################################
# PROJE GÖREVLERİ
#############################################

#############################################
# GÖREV 1: Aşağıdaki soruları yanıtlayınız.
#############################################


In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [8]:
#############GÖREV 1##################
# Soru 1: persona.csv dosyasını okutunuz ve veri seti ile ilgili genel bilgileri gösteriniz.
persona = pd.read_csv(r"C:\Users\1999m\Desktop\Yeni klasör (2)\persona.csv")
df = persona.copy()
df.head()

,PRICE,SOURCE,SEX,COUNTRY,AGE
0,39,android,male,bra,17
1,39,android,male,bra,17
2,49,android,male,bra,17
3,29,android,male,tur,17
4,49,android,male,tur,17


In [9]:
# Soru 2: Kaç unique SOURCE vardır? Frekansları nedir?
df["SOURCE"].nunique()  

2

In [10]:
# Soru 3: Kaç unique PRICE vardır?
df["PRICE"].nunique()

6

In [11]:
# Soru 4: Hangi PRICE'dan kaçar tane satış gerçekleşmiş?
df["PRICE"].value_counts()

29    1305
39    1260
49    1031
19     992
59     212
9      200
Name: PRICE, dtype: int64

In [12]:
# Soru 5: Hangi ülkeden kaçar tane satış olmuş?
df["COUNTRY"].value_counts()

usa    2065
bra    1496
deu     455
tur     451
fra     303
can     230
Name: COUNTRY, dtype: int64

In [14]:
# Soru 6: Ülkelere göre satışlardan toplam ne kadar kazanılmış?
df.groupby("COUNTRY").agg({"PRICE": "sum"}).sort_values("PRICE", ascending=False)

,PRICE
COUNTRY,
usa,70225
bra,51354
tur,15689
deu,15485
fra,10177
can,7730


In [15]:
# Soru 7: SOURCE türlerine göre göre satış sayıları nedir?
df.groupby("SOURCE").agg({"PRICE": "count"}).sort_values("PRICE", ascending=False)

,PRICE
SOURCE,
android,2974
ios,2026


In [16]:
# Soru 8: Ülkelere göre PRICE ortalamaları nedir?
df.groupby("COUNTRY").agg({"PRICE": "mean"}).sort_values("PRICE", ascending=False)

,PRICE
COUNTRY,
tur,34.787140
bra,34.327540
deu,34.032967
usa,34.007264
can,33.608696
fra,33.587459


In [17]:
# Soru 9: SOURCE'lara göre PRICE ortalamaları nedir?
df.pivot_table(values="PRICE", index="SOURCE", aggfunc="mean")

,PRICE
SOURCE,
android,34.174849
ios,34.069102


In [18]:
# Soru 10: COUNTRY-SOURCE kırılımında PRICE ortalamaları nedir?
df.pivot_table(values="PRICE", index="COUNTRY", columns="SOURCE", aggfunc="mean")

SOURCE,android,ios
COUNTRY,,
bra,34.387029,34.222222
can,33.330709,33.951456
deu,33.869888,34.268817
fra,34.312500,32.776224
tur,36.229437,33.272727
usa,33.760357,34.371703


In [23]:
#############################################
# GÖREV 2: COUNTRY, SOURCE, SEX, AGE kırılımında ortalama kazançlar nedir?
#############################################
agg_df = df.pivot_table(
    values="PRICE", index=["COUNTRY", "SOURCE", "SEX", "AGE"], aggfunc="mean")
agg_df

PRICE
COUNTRY SOURCE  SEX    AGE           
bra     android female 15   38.714286
                       16   35.944444
                       17   35.666667
                       18   32.255814
                       19   35.206897
...                               ...
usa     ios     male   42   30.250000
                       50   39.000000
                       53   34.000000
                       55   29.000000
                       59   46.500000

[348 rows x 1 columns]

In [25]:
#############################################
# GÖREV 3: Çıktıyı PRICE'a göre sıralayınız.
#############################################
# Önceki sorudaki çıktıyı daha iyi görebilmek için sort_values metodunu azalan olacak şekilde PRICE'a uygulayınız.
# Çıktıyı agg_df olarak kaydediniz.
agg_df = agg_df.sort_values("PRICE", ascending=False)
agg_df

PRICE
COUNTRY SOURCE  SEX    AGE       
bra     android male   46    59.0
fra     android female 24    59.0
usa     android male   36    59.0
        ios     male   32    54.0
        android female 47    49.0
...                           ...
        ios     female 38    19.0
                       30    19.0
can     android female 27    19.0
fra     android male   18    19.0
deu     android male   26     9.0

[348 rows x 1 columns]

In [27]:
#############################################
# GÖREV 4: Indekste yer alan isimleri değişken ismine çeviriniz.
#############################################
# Üçüncü sorunun çıktısında yer alan PRICE dışındaki tüm değişkenler index isimleridir.
# Bu isimleri değişken isimlerine çeviriniz.
# İpucu: reset_index()
# agg_df.reset_index(inplace=True)
agg_df.reset_index(inplace=True)
agg_df

,index,COUNTRY,SOURCE,SEX,AGE,PRICE
0,0,bra,android,male,46,59.0
1,1,fra,android,female,24,59.0
2,2,usa,android,male,36,59.0
3,3,usa,ios,male,32,54.0
4,4,usa,android,female,47,49.0
...,...,...,...,...,...,...
343,343,usa,ios,female,38,19.0
344,344,usa,ios,female,30,19.0
345,345,can,android,female,27,19.0
346,346,fra,android,male,18,19.0


In [28]:
#############################################
# GÖREV 5: AGE değişkenini kategorik değişkene çeviriniz ve agg_df'e ekleyiniz.
#############################################
# Age sayısal değişkenini kategorik değişkene çeviriniz.
# Aralıkları ikna edici olacağını düşündüğünüz şekilde oluşturunuz.
# Örneğin: '0_18', '19_23', '24_30', '31_40', '41_70'
age_labels = ["0-18", "19-23", "24-30", "31-40", "41-70"]
agg_df["age_group"] = pd.cut(
    agg_df["AGE"], bins=[0, 18, 23, 30, 40, 70], labels=age_labels
)
agg_df.head()

,index,COUNTRY,SOURCE,SEX,AGE,PRICE,age_group
0,0,bra,android,male,46,59.0,41-70
1,1,fra,android,female,24,59.0,24-30
2,2,usa,android,male,36,59.0,31-40
3,3,usa,ios,male,32,54.0,31-40
4,4,usa,android,female,47,49.0,41-70


In [31]:
#############################################
# GÖREV 6: Yeni level based müşterileri tanımlayınız ve veri setine değişken olarak ekleyiniz.
#############################################
# customers_level_based adında bir değişken tanımlayınız ve veri setine bu değişkeni ekleyiniz.
# Dikkat!
# list comp ile customers_level_based değerleri oluşturulduktan sonra bu değerlerin tekilleştirilmesi gerekmektedir.
# Örneğin birden fazla şu ifadeden olabilir: USA_ANDROID_MALE_0_18
# Bunları groupby'a alıp price ortalamalarını almak gerekmektedir.

# Müşteri özelliklerinden customers_level_based'i oluşturuyoruz
agg_df["customers_level_based"] = (
    agg_df["COUNTRY"]
    + "_"
    + agg_df["SOURCE"]
    + "_"
    + agg_df["SEX"]
    + "_"
    + pd.cut(agg_df["AGE"], bins=[0, 18, 23, 30, 40, 70], labels=age_labels).astype(str)
)

# customers_level_based değerlerini tekilleştiriyoruz
agg_df["customers_level_based"] = (
    agg_df.groupby("customers_level_based")["PRICE"]
    .transform("mean")
    .sort_values(ascending=False)
    .index
)
agg_df.head()

,index,COUNTRY,SOURCE,SEX,AGE,PRICE,age_group,customers_level_based
0,0,bra,android,male,46,59.0,41-70,1
1,1,fra,android,female,24,59.0,24-30,249
2,2,usa,android,male,36,59.0,31-40,13
3,3,usa,ios,male,32,54.0,31-40,23
4,4,usa,android,female,47,49.0,41-70,18


In [32]:
#############################################
# GÖREV 7: Yeni müşterileri (USA_ANDROID_MALE_0_18) segmentlere ayırınız.
#############################################
# PRICE'a göre segmentlere ayırınız,
# segmentleri "SEGMENT" isimlendirmesi ile agg_df'e ekleyiniz,
# segmentleri betimleyiniz,

# PRICE'a göre segmentlere ayırma
agg_df["SEGMENT"] = pd.qcut(agg_df["PRICE"], q=4, labels=["D", "C", "B", "A"])

# Segmentleri betimleme
segment_summary = agg_df.groupby("SEGMENT").agg(
    {"PRICE": ["mean", "min", "max", "count"]}
)
agg_df.head()

,index,COUNTRY,SOURCE,SEX,AGE,PRICE,age_group,customers_level_based,SEGMENT
0,0,bra,android,male,46,59.0,41-70,1,A
1,1,fra,android,female,24,59.0,24-30,249,A
2,2,usa,android,male,36,59.0,31-40,13,A
3,3,usa,ios,male,32,54.0,31-40,23,A
4,4,usa,android,female,47,49.0,41-70,18,A
